In [1]:
# %load /home/oweser/.startup1
import pandas as pd
import chemcoord as cc
from chemcoord.exceptions import InvalidReference
import functools
from functools import partial
import numpy as np
from chemcoord.exceptions import InvalidReference
from numba import jit
import pytest
from contextlib import contextmanager
%load_ext line_profiler
import sys
# unless specified else, all calculations are carried out in SI units

In [2]:
molecule = cc.Cartesian.read_xyz('water.xyz', start_index=1)
molecule = molecule - molecule.loc[5, ['x', 'y', 'z']]

zmolecule = molecule.get_zmat()
c_table = zmolecule.loc[:, ['b', 'a', 'd']]
c_table.loc[6, ['a', 'd']] = [2, 1]
zmolecule1 = molecule.get_zmat(construction_table=c_table)
zmolecule2 = zmolecule1.copy()
zmolecule3 = molecule.get_zmat(construction_table=c_table)

In [3]:
molecule.view()

### Modifications on zmolecule1

In [14]:
angle_before_assignment = zmolecule1.loc[4, 'angle']

In [15]:
zmolecule1.safe_loc[4, 'angle'] = 180

/home/oweser/code/git/chemcoord/src/chemcoord/internal_coordinates/_zmat_class_core.py:368: UserWarning: For the dihedral reference of atom 5 the dummy atom 7 was inserted
  warnings.warn(give_message(i=i, dummy_d=dummy_d), UserWarning)
/home/oweser/code/git/chemcoord/src/chemcoord/internal_coordinates/_zmat_class_core.py:368: UserWarning: For the dihedral reference of atom 6 the dummy atom 8 was inserted
  warnings.warn(give_message(i=i, dummy_d=dummy_d), UserWarning)


In [6]:
zmolecule1.safe_loc[5, 'dihedral'] = 90

In [8]:
zmolecule1.safe_loc[4, 'angle'] = angle_before_assignment

In [9]:
xyz1 = zmolecule1.give_cartesian()

### Modifications on zmolecule2

In [10]:
zmolecule2.safe_loc[5, 'dihedral'] = 90

In [11]:
xyz2 = zmolecule2.give_cartesian()
# xyz2.view()

### Comparison

In [12]:
cc.xyz_functions.allclose(xyz1, xyz2)

True

In [16]:
zmolecule1._metadata

{'DummyManipulation_allowed': True,
 'has_dummies': {5: {'actual_d': 1, 'dummy_d': 7},
  6: {'actual_d': 1, 'dummy_d': 8}},
 'last_valid_cartesian':   atom         x         y             z
 1    O -3.758602 -0.500000 -5.042840e-01
 2    H -3.000000 -0.500000  0.000000e+00
 3    H -3.498147 -0.500000 -1.377177e+00
 4    O -1.041952 -0.500000  1.301621e+00
 7    X -3.000000  0.500000 -6.753223e-16
 5    H -0.157780 -0.500000  1.082490e+00
 8    X -3.000000  0.500000 -6.753223e-16
 6    H -0.540245  0.259636  1.333583e+00}

## Contextmanager

In [17]:
with cc.DummyManipulation(zmolecule3, False):
    try:
        zmolecule3.safe_loc[4, 'angle'] = 180
    except InvalidReference as e:
        e.already_built_cartesian.view()
        test = e.zmat_after_assignment._insert_dummy_zmat(e)

/home/oweser/code/git/chemcoord/src/chemcoord/internal_coordinates/_zmat_class_core.py:368: UserWarning: For the dihedral reference of atom 5 the dummy atom 7 was inserted
  warnings.warn(give_message(i=i, dummy_d=dummy_d), UserWarning)
/home/oweser/code/git/chemcoord/src/chemcoord/internal_coordinates/_zmat_class_core.py:368: UserWarning: For the dihedral reference of atom 6 the dummy atom 8 was inserted
  warnings.warn(give_message(i=i, dummy_d=dummy_d), UserWarning)


In [18]:
zmolecule3

,atom,b,bond,a,angle,d,dihedral
1,O,$\vec{0}$,3.825100,$\vec{e_z}$,97.575672,$\vec{e_x}$,187.577464
2,H,1,0.910922,$\vec{e_z}$,13.716615,$\vec{e_x}$,213.402944
3,H,1,0.910922,2,107.000024,$\vec{e_x}$,347.276217
4,O,2,2.351206,1,132.466298,3,343.244987
5,H,4,0.910922,2,132.466298,1,180.000000
6,H,4,0.910922,2,118.561123,1,18.293238


In [18]:
test[~test.index.isin(zmolecule3.index)]

,atom,b,bond,a,angle,d,dihedral
7,X,-9223372036854775808,2.604064,-9223372036854775805,107.83188,-9223372036854775807,184.89941
8,X,-9223372036854775808,2.604064,-9223372036854775805,107.83188,-9223372036854775807,184.89941
